In [1]:
from subpred.protein_go_datasets import get_transmembrane_transporter_dataset, get_stats, get_uniprot_go_dataset

cd-hit: clustered 347 sequences into 291 clusters at threshold 50
cd-hit: clustered 347 sequences into 318 clusters at threshold 70
cd-hit: clustered 347 sequences into 339 clusters at threshold 90
cd-hit: clustered 347 sequences into 347 clusters at threshold 100


n_transporters  \
swissprot_reviewed has_gene_name go_evidence   protein_existence_evidence clustering                   
True               True          computational protein_level              50                     222   
                                                                          70                     246   
                                                                          90                     266   
                                                                          100                    274   
                                                                          None                   274   
                                 experiment    protein_level              50                     263   
                                                                          70                     287   
                                                                          90                     308   
                                                                          100                    316   
                                                                          None                   316   

                                                                                      n_terms  
swissprot_reviewed has_gene_name go_evidence   protein_existence_evidence clustering           
True               True          computational protein_level              50              130  
                                                                          70              131  
                                                                          90              131  
                                                                          100             131  
                                                                          None            131  
                                 experiment    protein_level              50              280  
                                                                          70              282  
                                                                          90              283  
                                                                          100             283  
                                                                          None            283

In [8]:
dataset_yeast = get_transmembrane_transporter_dataset(  # yeast
    organism_ids={559292},
    swissprot_only=True,
    datasets_path="../data/datasets/",
    exclude_iea_go_terms=False,
    max_sequence_evidence_code=1,
    remove_proteins_without_gene_names=True,
)
get_stats(*dataset_yeast)

cd-hit: clustered 347 sequences into 291 clusters at threshold 50
cd-hit: clustered 347 sequences into 318 clusters at threshold 70
cd-hit: clustered 347 sequences into 339 clusters at threshold 90
cd-hit: clustered 347 sequences into 347 clusters at threshold 100


n_transporters  \
swissprot_reviewed has_gene_name go_evidence   protein_existence_evidence clustering                   
True               True          computational protein_level              50                     222   
                                                                          70                     246   
                                                                          90                     266   
                                                                          100                    274   
                                                                          None                   274   
                                 experiment    protein_level              50                     263   
                                                                          70                     287   
                                                                          90                     308   
                                                                          100                    316   
                                                                          None                   316   

                                                                                      n_terms  
swissprot_reviewed has_gene_name go_evidence   protein_existence_evidence clustering           
True               True          computational protein_level              50              130  
                                                                          70              131  
                                                                          90              131  
                                                                          100             131  
                                                                          None            131  
                                 experiment    protein_level              50              280  
                                                                          70              282  
                                                                          90              283  
                                                                          100             283  
                                                                          None            283

In [9]:
dataset_meta = get_transmembrane_transporter_dataset(  # meta
    organism_ids=[3702, 9606, 83333, 559292],
    swissprot_only=True,
    datasets_path="../data/datasets/",
    exclude_iea_go_terms=True,
    max_sequence_evidence_code=1,
    remove_proteins_without_gene_names=True,
)
get_stats(*dataset_meta)

cd-hit: clustered 2260 sequences into 1741 clusters at threshold 50
cd-hit: clustered 2260 sequences into 2060 clusters at threshold 70
cd-hit: clustered 2260 sequences into 2224 clusters at threshold 90
cd-hit: clustered 2260 sequences into 2256 clusters at threshold 100


n_transporters  \
swissprot_reviewed has_gene_name go_evidence protein_existence_evidence clustering                   
True               True          experiment  protein_level              50                    1741   
                                                                        70                    2060   
                                                                        90                    2224   
                                                                        100                   2256   
                                                                        None                  2260   

                                                                                    n_terms  
swissprot_reviewed has_gene_name go_evidence protein_existence_evidence clustering           
True               True          experiment  protein_level              50              718  
                                                                        70              740  
                                                                        90              751  
                                                                        100             751  
                                                                        None            751

In [3]:
dataset_athaliana = get_transmembrane_transporter_dataset(  # athaliana
    organism_ids=[3702],
    swissprot_only=True,
    datasets_path="../data/datasets/",
    exclude_iea_go_terms=True,
    max_sequence_evidence_code=1,
    remove_proteins_without_gene_names=True,
)
get_stats(*dataset_athaliana)

cd-hit: clustered 420 sequences into 276 clusters at threshold 50
cd-hit: clustered 420 sequences into 344 clusters at threshold 70
cd-hit: clustered 420 sequences into 403 clusters at threshold 90
cd-hit: clustered 420 sequences into 419 clusters at threshold 100


n_transporters  \
swissprot_reviewed has_gene_name go_evidence protein_existence_evidence clustering                   
True               True          experiment  protein_level              50                     276   
                                                                        70                     344   
                                                                        90                     403   
                                                                        100                    419   
                                                                        None                   420   

                                                                                    n_terms  
swissprot_reviewed has_gene_name go_evidence protein_existence_evidence clustering           
True               True          experiment  protein_level              50              264  
                                                                        70              281  
                                                                        90              288  
                                                                        100             288  
                                                                        None            288